## Programación de los sub modelos del Modelo de Estructura Urbana y Regional.

A continuación, se encuentra la programación de los 3 submodelos del Modelo de estructura urbana y regional: Submodelo de repartición de construcción regional, Submodelo de usos del suelo y submodelo de localización de hogares.

## Submodelo de repartición de construcción regional.

El primer modelo a programar es el de repartición de construcción regional, el cual 
tiene en cuenta tanto la dinámica de construcción previa de cada unidad espacial como las restricciones 
normativas que delimitan el crecimiento en la región 

El primer paso a realizar es importar las librerías que permitirán el tratamiento de los datos de entrada y los ejercicios propuestos.

In [ ]:
import statsmodels.api as sm
from patsy import dmatrices
import statsmodels.api as st
from patsy import dmatrices
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import patsy as pt
from statsmodels.genmod.generalized_estimating_equations import GEE
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)
from statsmodels.genmod.families import Poisson

El siguiente paso consiste en copiar la ubicación donde se encuentran los archivos de entrada del modelo, preferiblemente guardar todos los archivos de entrada de los submodelos en la misma carpeta contenedora.

In [ ]:
os.chdir(r'C:\Users\david\OneDrive\Escritorio\')

Después, llamar los archivos de entrada del modelo de repartición de construcción regional.

In [ ]:
ipath1 = 'tiempo_modelo.xlsx'
ipath2 = 'betas_modelo.xlsx'
ipath3 = 'matriz_contiguidad.xlsx'

    
    
llenado = pd.read_excel(ipath1)
delta_llenado1 = pd.read_excel(ipath1)
llenado1_cr = pd.read_excel(ipath1)
llenado_sr = pd.read_excel(ipath1)
llenado_cr = pd.read_excel(ipath1)
proporcion_cr = pd.read_excel(ipath1)
delta_llenado = pd.read_excel(ipath1)
proporcion = pd.read_excel(ipath1)
llenado1 = pd.read_excel(ipath1)
repartidor  = pd.read_excel(ipath1)
repartidor1 = pd.read_excel(ipath1)
df = pd.read_excel(ipath2)
matriz_con = pd.read_excel(ipath3)


Se generará una matriz tamaño 149x35 que contiene los metros cuadrados proyectados por año para cada unidad espacial, que tiene en cuenta el componente normativo. Si el stock de metros cuadrados proyectados se encuentra por debajo de la restricción se asume que es posible construir en esa unidad espacial en ese año; de lo contrario se asumirá que el stock permanecerá constante y será igual a la restricción normativa.

In [ ]:
for x in range(34): 
    s=2017+x
    for i in range(len(llenado)):
        if df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i]))) < (df['Area maxima adicional'][i]+df['Area 2017'][i]):
            llenado[s][i] = df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i])))
        else:
            llenado[s][i] = df['Area maxima adicional'][i]+df['Area 2017'][i]

Después se generará una matriz de las mismas dimensiones en la cual si la unidad espacial llego a su área máxima construida permitida para algun **t** se devolverá el valor proyectado menos la restricción, es decir la cantidad de metros cuadrados que se habían proyectado para dicha unidad espacial pero que no se efectuaron por la condición normativa; de lo contrario la función devolverá un 0.

In [ ]:
for x in range(34):
    s=2017+x
    for i in range (len(repartidor)):
        if df['Area maxima adicional'][i]+df['Area 2017'][i]-1 < llenado[s][i] <df['Area maxima adicional'][i]+df['Area 2017'][i]+1:
            repartidor[s][i] = (df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i]))) - df['Area maxima adicional'][i]-df['Area 2017'][i])
        else:
            repartidor[s][i] = 0

Se procede a eliminar y organizar el data frame y a eliminar las columnas que no pertenecen a la estimación, con el fin que al trasponer la matriz quede con las dimensiones requeridas para poder realizar el producto matricial

In [ ]:
repartidor.sort_values(by='CodSimulador')           
repartidor = repartidor.set_index('CodSimulador')
repartidor = repartidor.drop('Identifica', 1)  
matriz_con = matriz_con.set_index('Etiquetas de fila')
matriz_con_t=np.transpose(matriz_con)
mmult= []

Se procede a realizar el producto matricial entre la matriz de contiguidad generada con anterioridad y la matriz transpuesta que reparte los m2 de aquellas unidades espaciales que ya superaron su normativa de construcción, esto genera una matriz que contiene la cantidad de metros cuadrados asignados a las unidades contiguas.

In [ ]:
A=np.array(matriz_con_t)
B=np.array(repartidor)
mmult = np.dot(A,B)
asignaciones = pd.DataFrame(mmult)
asignaciones.rename(columns={0: 2017, 1: 2018, 2: 2019, 3: 2020, 4: 2021, 5:2022,  6: 2023, 7:2024, 8: 2025,  9:2026,  10:2027,   11:2028,   12:2029, 13:2030,  14:2031,  15:2032,  16:2033,   17:2034,  18:2035,  19:2036,  20:2037,  21:2038,  22:2039,  23:2040,  24:2041,  25:2042,  26:2043,  27:2044,  28:2045,  29:2046,  30:2047, 31:2048,  32:2049, 33:2050}, inplace=True)

Se realizar la suma entre la matriz de llenado que se creo al principio y la resultante del producto matricial anteriormente mencionado, esta matriz ocasiona que aquellas unidades espaciales que son contiguas a aquellas que llegaron a su normativa de manera rápida tengan una mayor proyección de construcción ya que reciben dichos m2 de sus contiguas, al igual que en la primera matriz si se cumple que una unidad espacial llego a su máximo normativo no se le asignan más m2

In [ ]:
for x in range(34):
    s=2017+x
    for i in range(len(llenado)):
        if df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i]))) + asignaciones[s][i] < (df['Area maxima adicional'][i]+df['Area 2017'][i]):
            llenado1[s][i] = df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i]))) + asignaciones[s][i]
        else:
            llenado1[s][i] = df['Area maxima adicional'][i]+df['Area 2017'][i]

Para encontrar la cantidad de m2 que se asignan por año se calcula la primera diferencia del proceso ya que en las anteriores matrices se encuentra el stock de construcción, sí las unidades espaciales han llegado a su máximo normativo el cambio de construcción será igual a 0.

In [ ]:
for x in range(34):
    s=2017+x
    for i in range (len(delta_llenado)): 
        if s==2017:
            delta_llenado1[s][i]=0
        else:
            delta_llenado1[s][i]= llenado1[s][i]-llenado1[s-1][i]

Se procede a calcular los m2 que se proyectan cada año para la región, para ello se suman todas las filas para cada año.

In [ ]:
suma1 = delta_llenado1.sum(axis=0)

df_cr = []
df_cr = pd.DataFrame(data = suma1.values)
df_cr_t =df_cr.T
df_cr_t.rename(columns={3: 2017, 4: 2018, 5: 2019, 6: 2020, 7: 2021, 8:2022,  9: 2023, 10:2024, 11: 2025,  12:2026,  13:2027,   14:2028,   15:2029, 16:2030,  17:2031,  18:2032,  19:2033,   20:2034,  21:2035,  22:2036,  23:2037,  24:2038,  25:2039,  26:2040,  27:2041,  28:2042,  29:2043,  30:2044,  31:2045,  32:2046,  33:2047, 34:2048,  35:2049, 36:2050}, inplace=True)

Por último en este submodelo se calculan los porcentajes de repartición para cada año, de tal manera que la suma de todas las columnas es igual a 1, esta matriz permite identificar cuantos m2 construidos para cada año se le asignan a cada una de las unidades espaciales del modelo, teniendo en cuenta la inercia de construcción existente y las condiciones normativas.

In [ ]:
for x in range(34):
    s=2017+x
    for i in range (len(proporcion_cr)):
        if s==2017:
            proporcion_cr[s][i]=0
        else:
         proporcion_cr[s][i] = delta_llenado1[s][i]/df_cr_t[s-1]*10000
 

for x in range(34):
    s=2017+x
    for i in range (len(proporcion_cr)):
        proporcion_cr[s][i]= proporcion_cr[s][i]/100

Exportar las salidas del modelo a extensión .xlsx.

In [ ]:
llenado.to_excel(r'D:\documentos llenado\documentos llenado\llenado_1.xlsx')
llenado1.to_excel(r'D:\documentos llenado\documentos llenado\llenado_2.xlsx')
proporcion_cr.to_excel(r'D:\documentos llenado\documentos llenado\proporcion_cr.xlsx')

## Submodelo de usos del suelo.

El primer paso consiste en llamar los archivos de entrada del submodelo de usos.

In [ ]:
ipath4 = 'variables_llenado.xlsx'
ipath5 = 'coeficientes_llenado.xlsx'
ipath6 = 'probabilidades llenado.xlsx'

variables = pd.read_excel(ipath4)
coeficientes = pd.read_excel(ipath5)
probabilidades = pd.read_excel(ipath6)
prob_2018 = pd.read_excel(ipath6)
prob_2025 = pd.read_excel(ipath6)

Es necesario declarar la base de probabilidades como tipo flotante.

In [ ]:
probabilidades['p_residencial'] = probabilidades['p_residencial'].astype(float)
probabilidades['p_comercial'] = probabilidades['p_comercial'].astype(float)
probabilidades['p_industrial'] = probabilidades['p_industrial'].astype(float)
probabilidades['p_dotacional'] = probabilidades['p_dotacional'].astype(float)
probabilidades['p_agricola'] = probabilidades['p_agricola'].astype(float)

Calcular los numeradores de la ecuación asociada al modelo de probabilidad logístico, utilizando tanto los coeficientes previamente estimados como las variables del modelo.

In [ ]:
for n in range(len(probabilidades)):
    probabilidades['p_residencial'][n] =1
    
for n in range(len(probabilidades)):
    probabilidades['p_comercial'][n] = math.exp(coeficientes['coef51'][n]+coeficientes['coef52'][n]*variables['lacc_0_5'][n]+
                  coeficientes['coef53'][n]*variables['ln_dvias'][n])
    
for n in range(len(probabilidades)):
    probabilidades['p_industrial'][n] = math.exp(coeficientes['coef61'][n]+coeficientes['coef62'][n]*variables['lacc_0_5'][n]+
                  coeficientes['coef63'][n]*variables['ln_dvias'][n])

for n in range(len(probabilidades)):
    probabilidades['p_dotacional'][n] = math.exp(coeficientes['coef71'][n]+coeficientes['coef72'][n]*variables['lacc_0_5'][n]+
                  coeficientes['coef73'][n]*variables['ln_dvias'][n])

for n in range(len(probabilidades)):
    probabilidades['p_agricola'][n] = math.exp(coeficientes['coef81'][n]+coeficientes['coef82'][n]*variables['lacc_0_5'][n]+
                  coeficientes['coef83'][n]*variables['ln_dvias'][n])

Por construcción el denominador de la ecuación logística es igual a la suma de todos los denominadores de las categorías, por ello es necesario realizar dicha suma.

In [ ]:
suma3 = probabilidades.sum(axis=1)
for n in range(len(suma3)):
    suma3[n]=probabilidades['p_residencial'][n]+probabilidades['p_industrial'][n]+probabilidades['p_comercial'][n]+probabilidades['p_dotacional'][n]+probabilidades['p_agricola'][n]

Realizar el cociente entre el numerador y el denominador de la ecuación asociada.

In [ ]:
for n in range(len(probabilidades)):
    probabilidades['p_residencial'][n] = probabilidades['p_residencial'][n]/suma3[n]*100

for n in range(len(probabilidades)):
    probabilidades['p_comercial'][n] = probabilidades['p_comercial'][n]/suma3[n]*100
    
    
for n in range(len(probabilidades)):
    probabilidades['p_industrial'][n] = probabilidades['p_industrial'][n]/suma3[n]*100
    
    
for n in range(len(probabilidades)):
    probabilidades['p_dotacional'][n] = probabilidades['p_dotacional'][n]/suma3[n]*100
    

for n in range(len(probabilidades)):
    probabilidades['p_agricola'][n] = probabilidades['p_agricola'][n]/suma3[n]*100



Con el fin de asegurarnos que el proceso quedo correctamente realizado, realizamos la suma de todas las probabilidades para cada uno de los coeficientes, dicha suma tiene que ser igual a 1.

In [ ]:
suma4 = probabilidades.sum(axis=1)
for n in range(len(suma4)):
    suma4[n]=probabilidades['p_residencial'][n]+probabilidades['p_industrial'][n]+probabilidades['p_comercial'][n]+probabilidades['p_dotacional'][n]+probabilidades['p_agricola'][n]

Existen unidades espaciales a las cuales se le calculan probabilidades de usos, pero por restricciones normativas no puede construirse en ellas, por ello se ajusta el valor de la probabilidad de cada categoría a 0.

In [ ]:
probabilidades['p_residencial'][3] = 0
probabilidades['p_comercial'][3] = 0
probabilidades['p_industrial'][3] = 0
probabilidades['p_dotacional'][3] = 0
probabilidades['p_agricola'][3] = 0
probabilidades['p_residencial'][2] = 0
probabilidades['p_comercial'][2] = 0
probabilidades['p_industrial'][2] = 0
probabilidades['p_dotacional'][2] = 0
probabilidades['p_agricola'][2] = 0

Seleccionar la ubicación de salida y el nombre del archivo con las probabilidades estimadas.

In [ ]:
probabilidades.to_excel(r'D:\documentos llenado\documentos llenado\probabilidades_usos.xlsx')

## Submodelo de hogares

Declarar como un data frame la base del modelo.

In [ ]:
ipath_hogares = 'modelo_hogares.xlsx'
df_hogares = pd.read_excel(ipath_hogares)

Crear la constante del modelo de regresión

In [ ]:
df_hogares['intercept']= 1

Generar las matrices con las variables independientes y dependientes de los modelos.

In [ ]:
dep_hogares1 = df_hogares['mov_hogares1_upz']
dep_hogares2 = df_hogares['mov_hogares2_upz']
dep_hogares3 = df_hogares['mov_hogares3_upz']
indep_pois = df_hogares[['acc_3_10mil','m2_resid_mill','suelo_exp_mill','p_hog2_upz','p_hog3_upz','intercept']]

Estimación y predicción del modelo para la categoría de ingresos n1.

In [ ]:
poisson_hogares1_results = sm.GLM(dep_hogares1, indep_pois, family=sm.families.Poisson()).fit()
print(poisson_hogares1_results.summary())
df_hogares['predict_hogares1'] = poisson_hogares1_results.predict(indep_pois)
df_hogares['porc_hogares1']= df_hogares['predict_hogares1']/df_hogares['predict_hogares1'].sum()
df_hogares['porc_hogares1'].sum()

Estimación y predicción del modelo para la categoría de ingresos n2

In [ ]:
poisson_hogares2_results = sm.GLM(dep_hogares2, indep_pois, family=sm.families.Poisson()).fit()
print(poisson_hogares2_results.summary())
df_hogares['predict_hogares2'] = poisson_hogares2_results.predict(indep_pois)
df_hogares['porc_hogares2']= df_hogares['predict_hogares2']/df_hogares['predict_hogares2'].sum()
df_hogares['porc_hogares2'].sum()

Estimación y predicción del modelo para la categoría de ingresos n3.

In [ ]:
poisson_hogares3_results = sm.GLM(dep_hogares3, indep_pois, family=sm.families.Poisson()).fit()
print(poisson_hogares3_results.summary())
df_hogares['predict_hogares3'] = poisson_hogares3_results.predict(indep_pois)
df_hogares['porc_hogares3']= df_hogares['predict_hogares3']/df_hogares['predict_hogares3'].sum()
df_hogares['porc_hogares3'].sum()

Generar el data frame con las salidas de los tres modelos.

In [ ]:
salidas_hogares = df_hogares[['predict_hogares1','porc_hogares1','predict_hogares2','porc_hogares2','predict_hogares3','porc_hogares3']]

Exportar las salidas en formato xlsx.

In [ ]:
salidas_hogares.to_excel(r'C:\Users\HcK\Desktop\PythonMEU\salidas_hogares.xlsx')

## Integración repartidor y usos del suelo

La integración únicamente consiste en una función que devuelve para el año que se especifique la repartición de los metros cuadrados tanto para cada unidad espacial como para el tipo de uso. Por cada año se genera una matriz de tamaño 146x5.

In [ ]:
def year(z):

    prob_z = prob_2018 
    
    for n in range(len(prob_z)):
        prob_z['p_residencial'][n] = probabilidades['p_residencial'][n]*proporcion_cr[z][n]*100

    for n in range(len(prob_z)):
        prob_z['p_comercial'][n] = probabilidades['p_comercial'][n]*proporcion_cr[z][n]*100
    
    for n in range(len(prob_2018)):
        prob_z['p_industrial'][n] = probabilidades['p_industrial'][n]*proporcion_cr[z][n]*100

    for n in range(len(prob_2018)):
        prob_z['p_dotacional'][n] = probabilidades['p_dotacional'][n]*proporcion_cr[z][n]*100

    for n in range(len(prob_2018)):
        prob_z['p_agricola'][n] = probabilidades['p_agricola'][n]*proporcion_cr[z][n]*100
    
    prob_z['p_comercial'] = prob_z['p_comercial'].astype(float)
    prob_z['p_industrial'] = prob_z['p_industrial'].astype(float)
    prob_z['p_dotacional'] = prob_z['p_dotacional'].astype(float)
    prob_z['p_agricola'] = prob_z['p_agricola'].astype(float)
    prob_z['p_residencial'] = prob_z['p_residencial'].astype(float)

    for n in range(len(prob_2018)):
        prob_z['p_residencial'][n] =prob_z['p_residencial'][n]/10000

    for n in range(len(prob_2018)):
        prob_z['p_comercial'][n] =prob_z['p_comercial'][n]/10000
    
    for n in range(len(prob_2018)):
        prob_z['p_industrial'][n] =prob_z['p_industrial'][n]/10000
    
    for n in range(len(prob_2018)):
        prob_z['p_dotacional'][n] =prob_z['p_dotacional'][n]/10000

    for n in range(len(prob_2018)):
        prob_z['p_agricola'][n] =prob_z['p_agricola'][n]/10000
  
    prob_z.to_excel(r'D:\documentos llenado\documentos llenado\prob_'+str(z)+'.xlsx')    

year(2020)

## Analisís de sensibilidad.

A continuación se presenta el análisis de sensibilidad del sub modelo de repartición de construcción regional, el cual consta de 2 partes, la primera en un aumento de la capacidad normativa de las unidades espaciales del simulador, la segunda en cambios en la dinámica de llenado para cada una de las unidades espaciales.

In [ ]:
def aumento_area(f):
    
    disp_rural['prural'] = (disp_rural['prural']*f)
    for v in range(len(disp_rural)):
        df['Area maxima adicional'][v] = df['Area maxima adicional'][v]+ disp_rural['prural'][v]
    df.to_excel(r'C:\Users\david\OneDrive\Escritorio\programacion ocupacion y usos con archivos de entrada\nueva_area_adiciona'+str(f)+'.xlsx')
    
aumento_area (10000000)

In [ ]:
def sens(p):
    
    
    df['b2'] = (df['b2']*p)
    
    for x in range(34): 
        s=2017+x
        for i in range(len(llenado)):
            if df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i]))) < (df['Area maxima adicional'][i]+df['Area 2017'][i]):
                llenado[s][i] = df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i])))
            else:
             llenado[s][i] = df['Area maxima adicional'][i]+df['Area 2017'][i]


                     
    for x in range(34):
        s=2017+x
        for i in range (len(repartidor)):
            if df['Area maxima adicional'][i]+df['Area 2017'][i]-1 < llenado[s][i] <df['Area maxima adicional'][i]+df['Area 2017'][i]+1:
                repartidor[s][i] = (df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i]))) - df['Area maxima adicional'][i]-df['Area 2017'][i])
            else:
             repartidor[s][i] = 0


    A=np.array(matriz_con_t)
    B=np.array(repartidor)
    mmult = np.dot(A,B)
    asignaciones = pd.DataFrame(mmult)
    asignaciones.rename(columns={0: 2017, 1: 2018, 2: 2019, 3: 2020, 4: 2021, 5:2022,  6: 2023, 7:2024, 8: 2025,  9:2026,  10:2027,   11:2028,   12:2029, 13:2030,  14:2031,  15:2032,  16:2033,   17:2034,  18:2035,  19:2036,  20:2037,  21:2038,  22:2039,  23:2040,  24:2041,  25:2042,  26:2043,  27:2044,  28:2045,  29:2046,  30:2047, 31:2048,  32:2049, 33:2050}, inplace=True)


    for x in range(34):
        s=2017+x
        for i in range(len(llenado)):
            if df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i]))) + asignaciones[s][i] < (df['Area maxima adicional'][i]+df['Area 2017'][i]):
                llenado1[s][i] = df['b1'][i]/(1+math.exp(-df['b2'][i]*((s)-df['b3'][i]))) + asignaciones[s][i]
            else:
             llenado1[s][i] = df['Area maxima adicional'][i]+df['Area 2017'][i]


                
    for x in range(34):
        s=2017+x
        for i in range (len(delta_llenado)): 
            if s==2017:
                delta_llenado1[s][i]=0
            else:
             delta_llenado1[s][i]= llenado1[s][i]-llenado1[s-1][i]

    suma1 = delta_llenado1.sum(axis=0)

    df_cr = []
    df_cr = pd.DataFrame(data = suma1.values)
    df_cr_t =df_cr.T
    df_cr_t.rename(columns={3: 2017, 4: 2018, 5: 2019, 6: 2020, 7: 2021, 8:2022,  9: 2023, 10:2024, 11: 2025,  12:2026,  13:2027,   14:2028,   15:2029, 16:2030,  17:2031,  18:2032,  19:2033,   20:2034,  21:2035,  22:2036,  23:2037,  24:2038,  25:2039,  26:2040,  27:2041,  28:2042,  29:2043,  30:2044,  31:2045,  32:2046,  33:2047, 34:2048,  35:2049, 36:2050}, inplace=True)


    for x in range(34):
        s=2017+x
        for i in range (len(proporcion_cr)):
            if s==2017:
                proporcion_cr[s][i]=0
            else:
             proporcion_cr[s][i] = delta_llenado1[s][i]/df_cr_t[s-1]*10000
 

    for x in range(34):
        s=2017+x
        for i in range (len(proporcion_cr)):
            proporcion_cr[s][i]= proporcion_cr[s][i]/100

   
    proporcion_cr.to_excel(r'C:\Users\david\OneDrive\Escritorio\soportes contrato\SOPORTES MARZO  DFHL\SOPORTES MES DE MARZO ANUAR DFHL\pr_oporcion_cr_'+str(p)+'.xlsx')

sens(1.2)